# Quickstart

This notebook shows how to use PySTAC to read through the public Sentinel catalog and write a local version.

### Reading STAC

We can read a STAC catalog located at a publicly available endpoint:

In [1]:
from pystac import Catalog
from pystac.extensions.eo import EOExtension

cat = Catalog.from_file('./example-catalog/catalog.json')

There are a lot of items in this catalog; crawling through it all would take a significant amount of time. Here, we lean on the fact that [link resolution is lazy](https://pystac.readthedocs.io/en/latest/concepts.html#lazy-resolution-of-stac-objects) and get to a catalog that contains items:

In [2]:
while len(cat.get_item_links()) == 0:
    print('Crawling through {}'.format(cat))
    cat = next(cat.get_children())

Crawling through <Catalog id=landsat-stac-collection-catalog>


We can print some information about the catalog, including how many children it has:

In [3]:
print(cat.description)
print('Contains {} items.'.format(len(cat.get_item_links())))

Landat 8 imagery radiometrically calibrated and orthorectified using gound points and Digital Elevation Model (DEM) data to correct relief displacement.
Contains 4 items.


Let's grab the first item., check out it's cloud cover, and start exploring the assets.

In [4]:
item = next(cat.get_items())

You can access [common metadata](https://github.com/radiantearth/stac-spec/blob/v0.9.0/item-spec/common-metadata.md) fields through the common_metadata property of the item:

In [5]:
item.common_metadata.platform

'landsat-8'

This particular stac item implements the [eo extension](https://github.com/radiantearth/stac-spec/tree/v0.9.0/extensions/eo) extension. We can access the extension information through the "ext" property that's part of every Catalog, Collection and Item. For instance, to get the cloud cover, we can use:

In [6]:
eo_ext = EOExtension.ext(item)
eo_ext.cloud_cover

22

we can see the cloud cover is in it's appropriate key in the Item's properties:

In [7]:
item.properties['eo:cloud_cover']

22

if we want to set the cloud cover, we can do that through the extension as well:

In [9]:
eo_ext.cloud_cover = 42.0
item.properties['eo:cloud_cover']

42.0

We can access the item's assets through the assets property, which is a dictionary:

In [10]:
for asset_key in item.assets:
    asset = item.assets[asset_key]
    print('{}: {} ({})'.format(asset_key, asset.href, asset.media_type))

index: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/index.html (text/html)
thumbnail: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_thumb_large.jpg (image/jpeg)
B1: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B1.TIF (image/tiff)
B2: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B2.TIF (image/tiff)
B3: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B3.TIF (image/tiff)
B4: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B4.TIF (image/tiff)
B5: https://s3-us

We can use the `to_dict()` method to convert an Asset, or any PySTAC object, into a dictionary:

In [11]:
asset = item.assets['B3']
asset.to_dict()

{'href': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B3.TIF',
 'type': 'image/tiff',
 'title': 'Band 3 (green)',
 'eo:bands': [{'name': 'B3',
   'full_width_half_max': 0.06,
   'center_wavelength': 0.56,
   'common_name': 'green'}],
 'roles': []}

Here we use the eo extension to get the band information for the asset:

In [15]:
eo_asset_ext = EOExtension.ext(asset)
bands = eo_asset_ext.bands
bands

[<Band name=B3>]

In [16]:
bands[0].to_dict()

{'name': 'B3',
 'full_width_half_max': 0.06,
 'center_wavelength': 0.56,
 'common_name': 'green'}

### Writing a STAC

Let's walk the catalog again, but this time create local clones of the STAC object, so we can end up with a copy that we can save off to the local file system.

In [ ]:
import itertools

cat = Catalog.from_file('https://sentinel-stac.s3.amazonaws.com/catalog.json')

# Setup the root of our local STAC
local_root = cat.clone()
local_root.clear_children()

# Loop over catalogs and clone
curr_local_cat = local_root
while len(cat.get_item_links()) == 0:
    print('Crawling through {}'.format(cat))
    cat = next(cat.get_children())
    local_cat = cat.clone()
    local_cat.clear_children()
    curr_local_cat.add_child(local_cat)
    curr_local_cat = local_cat
    
# Clear the items from the last local catalog
curr_local_cat.clear_children()
curr_local_cat.clear_items()

# Take the first 5 items
items = itertools.islice(cat.get_items(), 5)

# Clone and add them to our local catalog
curr_local_cat.add_items([i.clone() for i in items])

Now that we have a smaller STAC, let's map over the items to reduce it even further by only including the thumbnail assets in our items:

In [ ]:
def item_mapper(item):
    thumbnail_asset = item.assets['thumbnail']
    
    new_assets = { 
        k:v 
        for k, v in item.assets.items()
        if k == 'thumbnail'
    }
    item.assets = new_assets
    return item

local_root_2 = local_root.map_items(item_mapper)
    

We can now normalize our catalog and save it somewhere local:

In [ ]:
!mkdir -p ./quickstart_stac

In [ ]:
local_root_2.normalize_hrefs('./quickstart_stac')

In [ ]:
from pystac import CatalogType

local_root_2.save(catalog_type=CatalogType.SELF_CONTAINED)

In [ ]:
local_root_2.describe()

In [ ]:
for item in local_root_2.get_all_items():
    print('Item {}:')
    print('  Assets: {}'.format(item.assets))

### Validating

If we have `jsonschema` installed, either manually or via installing PySTAC with the optional validation dependencies installed:

```
> pip install pystac[validation]
```

we can validate our STAC objects to ensure we didn't set any properties that break the spec:

In [ ]:
for root, _, items in local_root_2.walk():
    root.validate()
    for item in items:
        item.validate()

If we have set something wrong, we'll get a `STACValidationException` when we validate:

In [ ]:
item = next(local_root_2.get_all_items())
item.bbox = ['Not a valid bbox']
item.validate()